In [19]:
from google.colab import files

uploaded = files.upload()

Saving cbsa_data_realpageflag.csv to cbsa_data_realpageflag.csv


In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [29]:
df = pd.read_csv("cbsa_data_realpageflag.csv", low_memory=False)

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# high missing values
missing_threshold = 0.5 * len(df)
high_missing_cols = df.columns[df.isnull().sum() > missing_threshold].tolist()

# Identify low-variance features
low_variance_cols = df[numeric_cols].var()[df[numeric_cols].var() < 0.01].index.tolist()

# highly correlated features
correlation_matrix = df[numeric_cols].corr()
high_corr_pairs = correlation_matrix.abs().unstack().sort_values(ascending=False)
high_corr_pairs = high_corr_pairs[high_corr_pairs < 1].reset_index()
high_corr_pairs.columns = ["Feature_1", "Feature_2", "Correlation"]
high_corr_features = set(high_corr_pairs[high_corr_pairs["Correlation"] > 0.9]["Feature_1"])

# Don't remove MPF features yet
mpf_columns = [col for col in df.columns if "MPF" in col]

# Remove only non-MPF columns
columns_to_remove = list(set(high_missing_cols + low_variance_cols + list(high_corr_features)))
columns_to_remove = [col for col in columns_to_remove if col not in mpf_columns]

# Create cleaned dataset
df_cleaned = df.drop(columns=columns_to_remove, errors='ignore')

# Save
df_cleaned.to_csv("cbsa_data_final_cleaned.csv", index=False)

# Display removed columns summary
removed_cols_info = {
    "High Missing Values": high_missing_cols,
    "Low Variance": low_variance_cols,
    "Highly Correlated Features": list(high_corr_features)
}
removed_cols_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in removed_cols_info.items()]))

# Show the removed columns
print("Removed Columns Summary:")
print(removed_cols_df)


All MPF columns retained: True
Removed Columns Summary:
       High Missing Values             Low Variance Highly Correlated Features
0      MPF-HIST-CONC-RATIO         MPF-ANN-RENT-CHG        SH-HIST-ASK-RENT-PB
1      SH-HIST-ASK-RENT-PB      MPF-HIST-CONC-RATIO      MPF-HIST-AVG-ASK-RPSF
2          SH-HIST-RENT-PB    SH-HIST-CONC-RATIO-PB                   MPF-RENT
3    SH-HIST-CONC-RATIO-PB  SH-HIST-ANN-RENT-PB-CHG                   MPF-RPSF
4        SH-HIST-OCCUPANCY                      NaN    SH-HIST-AVG-ASK-RPSF-PB
5          SH-HIST-RPSF-PB                      NaN          MPF-HIST-ASK-RENT
6  SH-HIST-AVG-ASK-RPSF-PB                      NaN            SH-HIST-RENT-PB
7  SH-HIST-ANN-RENT-PB-CHG                      NaN            SH-HIST-RPSF-PB


In [30]:
df_cleaned.head()

,propertyId,propertyName,zipCode,address,city,state,averageSquareFootage,stories,stable,class,...,MPF-HIST-ASK-RENT,MPF-HIST-AVG-ASK-RPSF,MPF-HIST-CONC-RATIO,MPF-OCC,MPF-RENT,MPF-RPSF,data_index,cbsa_code,status,uses_realpage
0,7915,Warwick,79606,2400 Arrowhead Dr,Abilene,TX,956,2,stable,B,...,939.0,0.982,NaN,0.974,939.0,0.982,0,10180,success,0
1,9952,Mill Creek,79605,5249 US-277,Abilene,TX,953,2,stable,B,...,1032.0,1.082,NaN,0.994,1032.0,1.082,1,10180,success,0
2,10561,The Landing,79605,5450 Texas Ave,Abilene,TX,609,2,stable,B,...,893.0,1.466,NaN,0.970,893.0,1.466,2,10180,success,0
3,10762,Curry Junction,79606,3549 Curry Ln,Abilene,TX,819,2,stable,B,...,1021.0,1.245,NaN,0.972,1021.0,1.245,3,10180,success,0
4,11055,Indian Run,79606,3549 Cedar Run Rd,Abilene,TX,738,2,stable,B,...,912.0,1.235,NaN,0.934,912.0,1.235,4,10180,success,0


In [38]:


# Load dataset
df = pd.read_csv("cbsa_data_final_cleaned.csv", low_memory=False)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df_numeric = df[numeric_cols].copy()

if "MPF-RENT" not in df_numeric.columns:
    raise ValueError("MPF-RENT column is missing")

df_numeric.fillna(df_numeric.median(), inplace=True)

X = df_numeric.drop(columns=["MPF-RENT"], errors="ignore")
y = df_numeric["MPF-RENT"]

if X.empty or y.empty:
    raise ValueError("X or y is empty after preprocessing")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Regression Model
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const).fit()

# Model Summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:               MPF-RENT   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 2.665e+06
Date:                Mon, 10 Mar 2025   Prob (F-statistic):               0.00
Time:                        23:59:13   Log-Likelihood:            -1.4318e+05
No. Observations:               33600   AIC:                         2.864e+05
Df Residuals:                   33583   BIC:                         2.865e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -184.33

In [39]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Compute VIF
X_vif = sm.add_constant(X_train)
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Display VIF values
print(vif_data.sort_values(by="VIF", ascending=False))


                  Feature           VIF
0                   const  13277.748471
10  MPF-HIST-AVG-ASK-RPSF    157.714766
13               MPF-RPSF    148.678003
9       MPF-HIST-ASK-RENT      8.684156
2    averageSquareFootage      3.281505
1              propertyId      1.789149
7               yearBuilt      1.606027
11    MPF-HIST-CONC-RATIO      1.588995
12                MPF-OCC      1.457892
14             data_index      1.311653
3                 stories      1.305810
4                latitude      1.225063
6               unitcount      1.163557
5               longitude      1.123366
8        MPF-ANN-RENT-CHG      1.097844
16          uses_realpage      1.097536
15              cbsa_code      1.081347


In [40]:
# Drop the highly collinear feature
X_train = X_train.drop(columns=["MPF-HIST-AVG-ASK-RPSF"], errors="ignore")
X_test = X_test.drop(columns=["MPF-HIST-AVG-ASK-RPSF"], errors="ignore")

# Re-run regression
model_v2 = sm.OLS(y_train, sm.add_constant(X_train)).fit()

# Print updated regression summary
print(model_v2.summary())


                            OLS Regression Results                            
Dep. Variable:               MPF-RENT   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 3.306e+05
Date:                Tue, 11 Mar 2025   Prob (F-statistic):               0.00
Time:                        00:01:09   Log-Likelihood:            -1.7922e+05
No. Observations:               33600   AIC:                         3.585e+05
Df Residuals:                   33584   BIC:                         3.586e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -2.1792 